In [1]:
import glob
import numpy as np
import astropy.io.fits as fits

from   astropy.table import Table, join, unique, vstack
from   desiutil.log import get_logger

In [2]:
everest   = '/global/cfs/cdirs/desi/spectro/redux/everest/'

In [3]:
exposures = Table.read(everest + '/exposures-everest.fits', hdu=1)
exposures = exposures['EXPID', 'SEEING_ETC', 'AIRMASS', 'EBV', 'TRANSPARENCY_GFA', 'SEEING_GFA', 'SKY_MAG_AB_GFA', 'SKY_MAG_G_SPEC', 'SKY_MAG_R_SPEC', 'SKY_MAG_Z_SPEC', 'EFFTIME_SPEC']
exposures

EXPID,SEEING_ETC,AIRMASS,EBV,TRANSPARENCY_GFA,SEEING_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_SPEC
int32,float64,float32,float64,float64,float64,float64,float64,float64,float64,float32
67678,0.0,1.023959,0.05589795857667923,0.9421358004271907,2.3493915528589158,20.306343899519227,21.941855433300375,20.967777902066913,19.06485558344176,7.1578417
67679,0.0,0.0,0.0,0.9031746394430591,2.8709763698441533,20.253945455193083,21.990210548474533,21.005319539905468,19.11832218906547,0.0
67680,0.0,0.0,0.0,0.9527972869199486,2.183973198109514,20.033265558674195,21.974079414911984,21.00463565548948,19.136359781174068,0.0
67681,0.0,0.0,0.0,0.9507191598338869,2.2193172453875247,20.218061565792368,21.984141770313954,21.029691679011016,19.17373571059973,0.0
67682,0.0,0.0,0.0,0.9562083786822335,1.984629969729699,20.202504822354022,21.970702187540937,21.026567014552832,19.182125628343407,0.0
67683,0.0,0.0,0.0,0.9593816691164547,1.958987337182022,20.211071197447247,21.985165548596846,21.043199719879794,19.199415786170565,0.0
67684,0.0,0.0,0.0,0.978166395729116,1.9761185927634832,20.213012077803974,21.981274613992447,21.033829690050773,19.196835193759735,0.0
67685,0.0,0.0,0.0,0.9567281144246651,1.9975639097347049,20.21637489510896,22.010570813576145,21.055310412499367,19.21901700879002,0.0
67686,0.0,0.0,0.0,0.9574311971931043,2.159358609455105,20.22896099635877,22.018361094047165,21.067962842572513,19.239374632070636,0.0


In [4]:
# https://github.com/desihub/desispec/blob/42834c3e610fb98a4bb01fdb4599aa8816b2b338/bin/desi_tsnr_afterburner#L151
def read_gfa_data(gfa_proc_dir='/global/cfs/cdirs/desi/survey/GFA/'):
    '''
    Read the directory with the offline GFA data reduction (like /global/cfs/cdirs/desi/survey/GFA/),
    find the latest version of the table files for the various surveys, return the merged table.
    
    See documentation here https://desi.lbl.gov/trac/wiki/SurveyValidation/SV1/conditions/summary_files
    
    Args:
      gfa_proc_dir: str, directory path
    
    returns astropy.table.Table
    '''
    log = get_logger()
    
    tables=[]
    
    for survey in ["SV1","SV2","SV3"] :
        filenames=sorted(glob.glob("{}/offline_matched_coadd_ccds_{}-thru_????????.fits".format(gfa_proc_dir,survey)))
        if len(filenames)==0: 
            continue
        filename=filenames[-1]
        log.info(f"Reading {filename}")
        table=Table.read(filename,2)# HDU2 is average over frames during spectro exposure and median across CCDs
        tables.append(table)
    
    if len(tables)==0 :
        log=get_logger()
        mess="did not find any file offline_matched_coadd_ccds_*-thru_????????.fits in {}".format(gfa_proc_dir)
        log.critical(mess)
        raise RuntimeError(mess)
    
    table=vstack(tables)
    
    log.info(f'{len(table)} GFA table entries')
    
    return table

In [42]:
##  Note:  only SV3 in this directory. 
gfa_data  = read_gfa_data('/global/cfs/cdirs/desi/survey/GFA/')
gfa_data.dtype.names

INFO:<ipython-input-4-9a0afbf46f36>:23:read_gfa_data: Reading /global/cfs/cdirs/desi/survey/GFA/offline_matched_coadd_ccds_SV3-thru_20211103.fits
INFO:<ipython-input-4-9a0afbf46f36>:35:read_gfa_data: 2904 GFA table entries


('EXPID',
 'CUBE_INDEX',
 'NIGHT',
 'EXPTIME',
 'FNAME_RAW',
 'SKYRA',
 'SKYDEC',
 'PROGRAM',
 'MOON_ILLUMINATION',
 'MOON_ZD_DEG',
 'MOON_SEP_DEG',
 'KTERM',
 'FRACFLUX_NOMINAL_POINTSOURCE',
 'FRACFLUX_NOMINAL_ELG',
 'FRACFLUX_NOMINAL_BGS',
 'MJD',
 'FWHM_ASEC',
 'TRANSPARENCY',
 'SKY_MAG_AB',
 'FIBER_FRACFLUX',
 'FIBER_FRACFLUX_ELG',
 'FIBER_FRACFLUX_BGS',
 'AIRMASS',
 'RADPROF_FWHM_ASEC',
 'FIBERFAC',
 'FIBERFAC_ELG',
 'FIBERFAC_BGS',
 'MINCONTRAST',
 'MAXCONTRAST')

In [43]:
uids, cnts = np.unique(gfa_data['EXPID'].data, return_counts=True)
assert  cnts.max() == 1

In [44]:
gfa_data = gfa_data['EXPID', 'MOON_ILLUMINATION', 'MOON_SEP_DEG']

In [45]:
len(exposures), len(gfa_data)

(3912, 2904)

In [61]:
exposures = join(exposures, gfa_data, keys='EXPID', join_type='left')
exposures

EXPID,SEEING_ETC,AIRMASS,EBV,TRANSPARENCY_GFA,SEEING_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_SPEC,MOON_ILLUMINATION,MOON_SEP_DEG
int64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64
67678,0.0,1.023959,0.05589795857667923,0.9421358004271907,2.3493915528589158,20.306343899519227,21.941855433300375,20.967777902066913,19.06485558344176,7.1578417,--,--
67679,0.0,0.0,0.0,0.9031746394430591,2.8709763698441533,20.253945455193083,21.990210548474533,21.005319539905468,19.11832218906547,0.0,--,--
67680,0.0,0.0,0.0,0.9527972869199486,2.183973198109514,20.033265558674195,21.974079414911984,21.00463565548948,19.136359781174068,0.0,--,--
67681,0.0,0.0,0.0,0.9507191598338869,2.2193172453875247,20.218061565792368,21.984141770313954,21.029691679011016,19.17373571059973,0.0,--,--
67682,0.0,0.0,0.0,0.9562083786822335,1.984629969729699,20.202504822354022,21.970702187540937,21.026567014552832,19.182125628343407,0.0,--,--
67683,0.0,0.0,0.0,0.9593816691164547,1.958987337182022,20.211071197447247,21.985165548596846,21.043199719879794,19.199415786170565,0.0,--,--
67684,0.0,0.0,0.0,0.978166395729116,1.9761185927634832,20.213012077803974,21.981274613992447,21.033829690050773,19.196835193759735,0.0,--,--
67685,0.0,0.0,0.0,0.9567281144246651,1.9975639097347049,20.21637489510896,22.010570813576145,21.055310412499367,19.21901700879002,0.0,--,--
67686,0.0,0.0,0.0,0.9574311971931043,2.159358609455105,20.22896099635877,22.018361094047165,21.067962842572513,19.239374632070636,0.0,--,--


In [62]:
# Should apply sorted to glob.glob.
ntile = 10

tiles = [np.int(x.split('/')[-1]) for x in glob.glob('/global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/*')]
tiles = tiles[:ntile]
tiles

[338, 2977, 22109, 20393, 263, 1212, 334, 6373, 22982, 10315]

In [63]:
post_shutdown_coadd_fpaths = []

for tid in tiles:
    '''
    Retrieve coadd paths for all tiles & the latest thru_night.  
    '''
    
    fpaths = sorted(glob.glob('{}/tiles/cumulative/{:d}/*'.format(everest, tid)))
    
    if len(fpaths) > 1:
        nights = np.array([x.split('/')[-1] for x in fpaths]).astype(np.int)
        
        # print(np.sort(nights))
        
        assert np.all(nights == np.sort(nights))
     
    if len(fpaths) > 0:
        # We'll take the deepest only here. 
        fpath = fpaths[-1]
    
    else:
        print('Missing TILEID {:d}'.format(tid))
        
    # print(fpath)
    
    deepestnight = fpath.split('/')[-1]
    
    fpaths = sorted(glob.glob(fpath + '/' + 'coadd-?-{:d}-thru{}.fits'.format(tid, deepestnight)))
    '''
    for x in fpaths:
        print(x)
    '''
    post_shutdown_coadd_fpaths += [x for x in fpaths]

Missing TILEID 22109
Missing TILEID 6373
Missing TILEID 22982
Missing TILEID 10315


In [64]:
post_shutdown_coadd_fpaths[:12]

['/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-0-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-1-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-2-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-3-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-4-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-5-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-6-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-7-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-8-338-thru20210414.fits',
 '/global/

In [65]:
def process_coadd(coadd_fpath):
    '''
    Retrieve the input expids for each location on a given (tileid, thru_night).
    
    Note:  
        assuming input expids may differ by location due to quality cuts.  We 
        run round this later by processing simulatenously all locs with the same
        input expids.
    '''
    tileid     = coadd_fpath.split('/')[-3]
    thru_night = coadd_fpath.split('/')[-2]
 
    coadd      = Table.read(coadd_fpath, hdu='EXP_FIBERMAP')
    # coadd
    
    expids, cnts = np.unique(coadd['EXPID'], return_counts=True) 
    
    # print(len(expids))
    
    condition_cat = coadd['TARGETID', 'LOCATION']
    condition_cat = unique(condition_cat)
    condition_cat.sort('LOCATION')

    condition_cat['TILEID']     = tileid
    condition_cat['THRU_NIGHT'] = thru_night
    condition_cat['IN_EXPIDS']  = 'x' * 50
    
    locs           = np.unique(condition_cat['LOCATION'].data)

    for i, loc in enumerate(locs):
        coadd_loc  = coadd[(coadd['LOCATION'] == loc) & (coadd['FIBERSTATUS'] == 0)]

        # Here apply a fiber-status cut?        
        loc_expids = '-'.join(np.unique(coadd_loc['EXPID'].data).astype(str).tolist())
        
        condition_cat['IN_EXPIDS'][i] = loc_expids
          
        # print(i, loc_expids)
            
    return condition_cat

In [66]:
to_process    = post_shutdown_coadd_fpaths

# Add a codd with NEXP>1.
to_process   += ['/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative/10/20210503/coadd-0-10-thru20210503.fits']
to_process

['/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-0-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-1-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-2-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-3-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-4-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-5-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-6-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-7-338-thru20210414.fits',
 '/global/cfs/cdirs/desi/spectro/redux/everest//tiles/cumulative/338/20210414/coadd-8-338-thru20210414.fits',
 '/global/

In [67]:
condition_cat = [process_coadd(x) for x in to_process]
condition_cat = vstack(condition_cat)
condition_cat

TARGETID,LOCATION,TILEID,THRU_NIGHT,IN_EXPIDS
int64,int64,str5,str8,str50
39633136501393626,0,338,20210414,84821
39633136501393765,1,338,20210414,84821
39633136501393084,2,338,20210414,84821
39633136501394535,3,338,20210414,84821
39633136501393336,4,338,20210414,84821
39633132084791218,5,338,20210414,84821
39633132084790141,6,338,20210414,84821
39633132084789965,7,338,20210414,84821
616093884388213557,8,338,20210414,84821


In [68]:
processed     = []

update_cols   = list(exposures.dtype.names)
update_cols.remove('EXPID')
update_cols.remove('EFFTIME_SPEC')

for col in update_cols:
    condition_cat[col] = -99.

for row in condition_cat:
    if row['IN_EXPIDS'] not in processed:
        if len(row['IN_EXPIDS']) == 0:
            ##  Empty string, e.g. no input expids for negative targetids. 
            ##  Already defaults to -99. 
            continue
            
        expids        = np.array(row['IN_EXPIDS'].split('-'))
        
        expids        = np.array(row['IN_EXPIDS'].split('-')).astype(np.int)

        in_exposures  = exposures[np.isin(exposures['EXPID'].data, expids)]

        # print(expids)
        # print(in_exposures)
        
        mean_function = lambda x: np.average(x, weights=in_exposures['EFFTIME_SPEC'])
        
        in_exposures               = in_exposures.groups.aggregate(mean_function)
        in_exposures['IN_EXPIDS']  = row['IN_EXPIDS']
        
        del  in_exposures['EXPID']
        del  in_exposures['EFFTIME_SPEC']

        #  To be extra sure, we could include matches to TILEID and thru night.     
        to_update                  = condition_cat['IN_EXPIDS'] == row['IN_EXPIDS']
                
        for col in update_cols:
            condition_cat[col].data[to_update] = in_exposures[col].data[0]
        
        processed.append(row['IN_EXPIDS'])
    
        print(processed)
        
condition_cat.sort(['TILEID', 'THRU_NIGHT', 'LOCATION'])

['84821']
['84821', '97946-97947']
['84821', '97946-97947', '89875']
['84821', '97946-97947', '89875', '87254']
['84821', '97946-97947', '89875', '87254', '90406-90407']
['84821', '97946-97947', '89875', '87254', '90406-90407', '85512']
['84821', '97946-97947', '89875', '87254', '90406-90407', '85512', '87103-87104-87105']


In [69]:
exposures[np.isin(exposures['EXPID'].data, [87103, 87104, 87105, 84821])]

EXPID,SEEING_ETC,AIRMASS,EBV,TRANSPARENCY_GFA,SEEING_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_SPEC,MOON_ILLUMINATION,MOON_SEP_DEG
int64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64
84821,1.0429749488830566,1.021125,0.012264582328498363,0.6561105241107569,1.203344152557492,21.43737021851402,22.314847476552213,21.266628059680638,19.32414438845032,221.18857,0.095590784304443,118.64539430432494
87103,1.2934579849243164,1.278895,0.02310332842171192,1.0041470330873352,1.371289271220244,20.955625230974057,21.923068248181387,21.01512837361571,19.1370381394,557.0613,0.46312256854757394,159.15792995199234
87104,1.2934579849243164,1.337768,0.02310332842171192,1.0169399706068316,1.322608177199251,20.92023357471946,21.88635731486133,21.004868394711718,19.124612468336366,503.99792,0.46175035133055814,159.15790713220528
87105,1.2934579849243164,1.412697,0.02310332842171192,1.015957334617163,1.4874657628095873,20.918823289701905,21.860359708803173,20.983855227270976,19.072755906275862,273.28214,0.4604148142421023,159.15789044015847


In [70]:
condition_cat[::-1]

TARGETID,LOCATION,TILEID,THRU_NIGHT,IN_EXPIDS,SEEING_ETC,AIRMASS,EBV,TRANSPARENCY_GFA,SEEING_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,MOON_ILLUMINATION,MOON_SEP_DEG
int64,int64,str5,str8,str50,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
-1706286,9526,338,20210414,,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
39633114246415067,9525,338,20210414,84821,1.0429749488830566,1.0211249589920044,0.012264582328498363,0.6561105241107569,1.203344152557492,21.43737021851402,22.314847476552213,21.266628059680638,19.32414438845032,0.095590784304443,118.64539430432494
39633114246415512,9524,338,20210414,84821,1.0429749488830566,1.0211249589920044,0.012264582328498363,0.6561105241107569,1.203344152557492,21.43737021851402,22.314847476552213,21.266628059680638,19.32414438845032,0.095590784304443,118.64539430432494
39633114246416060,9523,338,20210414,84821,1.0429749488830566,1.0211249589920044,0.012264582328498363,0.6561105241107569,1.203344152557492,21.43737021851402,22.314847476552213,21.266628059680638,19.32414438845032,0.095590784304443,118.64539430432494
39633114246416725,9522,338,20210414,84821,1.0429749488830566,1.0211249589920044,0.012264582328498363,0.6561105241107569,1.203344152557492,21.43737021851402,22.314847476552213,21.266628059680638,19.32414438845032,0.095590784304443,118.64539430432494
39633114246417142,9521,338,20210414,84821,1.0429749488830566,1.0211249589920044,0.012264582328498363,0.6561105241107569,1.203344152557492,21.43737021851402,22.314847476552213,21.266628059680638,19.32414438845032,0.095590784304443,118.64539430432494
39633114246418257,9520,338,20210414,84821,1.0429749488830566,1.0211249589920044,0.012264582328498363,0.6561105241107569,1.203344152557492,21.43737021851402,22.314847476552213,21.266628059680638,19.32414438845032,0.095590784304443,118.64539430432494
39633114246418602,9519,338,20210414,84821,1.0429749488830566,1.0211249589920044,0.012264582328498363,0.6561105241107569,1.203344152557492,21.43737021851402,22.314847476552213,21.266628059680638,19.32414438845032,0.095590784304443,118.64539430432494
-1706278,9518,338,20210414,,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0


In [71]:
np.unique(condition_cat['IN_EXPIDS'].data)

array(['', '84821', '85512', '87103-87104-87105', '87254', '89875',
       '90406-90407', '97946-97947'], dtype='<U50')

In [72]:
condition_cat.write('/global/cscratch1/sd/mjwilson/trash/tsnr/condition_cat/test_cat.fits', format='fits', overwrite=True)

In [73]:
def test_coaddprop():
    test1=np.array([[1,2,  .5],[4,5,.5]])
    test2=np.array([[1,2,   1],[4,5, 0]])
    test3=np.array([[1,2,   0],[4,5, 1]])
    test4=np.array([[1,2, .75],[4,5, .25]])
    
    tests=[test1, test2, test3, test4]

    for i, test in enumerate(tests):
        print('\n\n-------  TEST {:d}  -------'.format(i))
        
        test = Table(test, names=['SKYMAG', 'MOONILL', 'EFFTIME_SPEC'])
        
        test.pprint()
        
        mean_function = lambda x: np.average(x, weights=test['EFFTIME_SPEC'])
        
        result = test.groups.aggregate(mean_function)

        print('\n\n-------  RESULT {:d}  -------'.format(i))
        
        result.pprint()

In [74]:
test_coaddprop()



-------  TEST 0  -------
SKYMAG MOONILL EFFTIME_SPEC
------ ------- ------------
   1.0     2.0          0.5
   4.0     5.0          0.5


-------  RESULT 0  -------
SKYMAG MOONILL EFFTIME_SPEC
------ ------- ------------
   2.5     3.5          0.5


-------  TEST 1  -------
SKYMAG MOONILL EFFTIME_SPEC
------ ------- ------------
     1       2            1
     4       5            0


-------  RESULT 1  -------
SKYMAG MOONILL EFFTIME_SPEC
------ ------- ------------
   1.0     2.0          1.0


-------  TEST 2  -------
SKYMAG MOONILL EFFTIME_SPEC
------ ------- ------------
     1       2            0
     4       5            1


-------  RESULT 2  -------
SKYMAG MOONILL EFFTIME_SPEC
------ ------- ------------
   4.0     5.0          1.0


-------  TEST 3  -------
SKYMAG MOONILL EFFTIME_SPEC
------ ------- ------------
   1.0     2.0         0.75
   4.0     5.0         0.25


-------  RESULT 3  -------
SKYMAG MOONILL EFFTIME_SPEC
------ ------- ------------
  1.75    2.75      

# Done.